In [152]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [153]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [154]:
df = pd.read_csv('../datasets/cosmetics.csv')

In [155]:
df.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1


In [156]:
import re
df['Name'] = df['Name'].str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', x))

In [157]:
df.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,LA MER,crme de la mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1
1,Moisturizer,SK-II,facial treatment essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,protini polypeptide cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0
3,Moisturizer,LA MER,the moisturizing soft cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1
4,Moisturizer,IT COSMETICS,your skin but better cc cream with spf,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1


In [158]:
df['Label'].unique()

array(['Moisturizer', 'Cleanser', 'Treatment', 'Face Mask', 'Eye cream',
       'Sun protect'], dtype=object)

In [159]:
from collections import Counter

c = Counter(''.join(df['Name']).split())
c.most_common(10)


[('spf', 212),
 ('eye', 177),
 ('spectrum', 115),
 ('broad', 114),
 ('face', 71),
 ('cream', 70),
 ('mask', 67),
 ('cleansing', 65),
 ('with', 62),
 ('skin', 58)]

Юзать признак name не надо 

In [160]:

from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np
# Предположим, что train_data['Description'] содержит 54,214 описаний
descriptions = df['Ingredients'].tolist()
model = SentenceTransformer('all-MiniLM-L6-v2')
# Определяем размер батча
embedding_batch_size = 128  # Можно изменить в зависимости от вашей системы

# Создаем список для хранения всех embedding
embeddings = []

# Используем tqdm для отображения прогресс-бара
for i in tqdm(range(0, len(descriptions), embedding_batch_size)):
    batch = descriptions[i:i + embedding_batch_size]
    batch_embeddings = model.encode(batch)
    embeddings.extend(batch_embeddings)

# Преобразуем embeddings в numpy array
embeddings = np.array(embeddings)


100%|██████████| 12/12 [01:43<00:00,  8.62s/it]


In [161]:

col = [f'{i+1}_feature' for i in range(embeddings.shape[1])]
# Преобразование разреженной матрицы в DataFrame
tfidf_df = pd.DataFrame(embeddings, columns=col)

# Объединение с исходным DataFrame
df = pd.concat([df, tfidf_df], axis=1)

df


,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,...,375_feature,376_feature,377_feature,378_feature,379_feature,380_feature,381_feature,382_feature,383_feature,384_feature
0,Moisturizer,LA MER,crme de la mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,...,0.002367,0.004616,0.120326,-0.063953,0.038606,-0.064546,-0.021260,0.019038,0.086378,0.068406
1,Moisturizer,SK-II,facial treatment essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,...,-0.019982,0.032502,0.080997,-0.064779,0.055741,-0.050618,0.016475,-0.033637,0.127291,0.059603
2,Moisturizer,DRUNK ELEPHANT,protini polypeptide cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,...,-0.064139,-0.002901,0.064029,-0.086758,0.029535,-0.089215,-0.041172,0.013025,0.102467,0.057022
3,Moisturizer,LA MER,the moisturizing soft cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,...,-0.023429,0.007025,0.083940,-0.082935,0.003240,-0.027722,-0.043147,0.015474,0.108699,0.096997
4,Moisturizer,IT COSMETICS,your skin but better cc cream with spf,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,...,-0.023434,0.019999,0.044448,-0.015759,-0.003949,-0.059385,-0.018034,0.018416,0.125612,0.070862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,Sun protect,KORRES,yoghurt nourishing fluid veil face sunscreen b...,35,3.9,"Water, Alcohol Denat., Potassium Cetyl Phospha...",1,1,1,1,...,-0.104420,-0.035631,0.058003,-0.010862,0.034614,-0.062651,-0.061828,0.038272,0.111087,0.055819
1468,Sun protect,KATE SOMERVILLE,daily deflector waterlight broad spectrum spf ...,48,3.6,"Water, Isododecane, Dimethicone, Butyloctyl Sa...",0,0,0,0,...,-0.051054,-0.030918,0.053407,-0.046240,0.042046,-0.070760,0.025236,0.021910,0.074194,0.071454
1469,Sun protect,VITA LIBERATA,self tan dry oil spf,54,3.5,"Water, Dihydroxyacetone, Glycerin, Sclerocarya...",0,0,0,0,...,-0.045909,-0.014331,0.054019,-0.049546,-0.017702,-0.057439,-0.054330,-0.008750,0.100576,0.082127
1470,Sun protect,ST. TROPEZ TANNING ESSENTIALS,pro light self tan bronzing mist,20,1.0,"Water, Dihydroxyacetone, Propylene Glycol, PPG...",0,0,0,0,...,-0.087394,-0.004894,0.023480,-0.026778,0.039399,-0.063929,0.030409,0.003675,0.123236,0.053287


In [162]:
"""
from sklearn.feature_extraction.text import TfidfVectorizer

# Инициализация TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(', '), max_features=100330)

# Преобразование ингредиентов в TF-IDF признаки
X = vectorizer.fit_transform(df['Ingredients'].fillna(''))

col = [f'{i+1}_feature' for i in range(X.shape[1])]
# Преобразование разреженной матрицы в DataFrame
tfidf_df = pd.DataFrame(X.toarray(), columns=col)

# Объединение с исходным DataFrame
df = pd.concat([df, tfidf_df], axis=1)

df
"""


"\nfrom sklearn.feature_extraction.text import TfidfVectorizer\n\n# Инициализация TfidfVectorizer\nvectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(', '), max_features=100330)\n\n# Преобразование ингредиентов в TF-IDF признаки\nX = vectorizer.fit_transform(df['Ingredients'].fillna(''))\n\ncol = [f'{i+1}_feature' for i in range(X.shape[1])]\n# Преобразование разреженной матрицы в DataFrame\ntfidf_df = pd.DataFrame(X.toarray(), columns=col)\n\n# Объединение с исходным DataFrame\ndf = pd.concat([df, tfidf_df], axis=1)\n\ndf\n"

In [163]:
df = df.drop(['Ingredients','Name'],axis=1)

In [164]:
target_columns = ['Combination','Dry','Normal','Oily','Sensitive']

In [165]:
bdf = BaseDatasetTransform(df,target=target_columns)

In [166]:
df,categorical_features = bdf.fit_transform()

Series([], dtype: int64)

В колонке(ах) ['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive'] нет пропущенных значений
-------------------------------------------
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['Label', 'Brand'], dtype='object')

Числовые колонки:
Index(['Price', 'Rank', 'Combination', 'Dry', 'Normal', 'Oily', 'Sensitive',
       '1_feature', '2_feature', '3_feature',
       ...
       '375_feature', '376_feature', '377_feature', '378_feature',
       '379_feature', '380_feature', '381_feature', '382_feature',
       '383_feature', '384_feature'],
      dtype='object', length=391)


In [167]:
X_train, X_test, y_train, y_test  = bdf.get_train_test_split()

Количество значений целевой переменной по категориям:
Количество значений комбинаций целевых переменных по категориям:
1_1_1_1_1    662
0_0_0_0_0    471
1_1_1_1_0    145
1_1_1_0_1     44
1_0_1_1_0     34
1_0_1_1_1     25
1_0_0_1_0     20
1_1_1_0_0     19
0_1_1_0_1      9
0_1_1_0_0      8
1_0_1_0_0      8
0_1_0_0_0      5
0_1_0_0_1      4
1_1_0_0_0      3
0_0_0_0_1      2
0_0_0_1_1      2
1_0_1_0_1      2
0_0_1_1_0      2
1_1_0_0_1      2
0_0_1_0_1      1
1_1_0_1_1      1
1_0_0_1_1      1
0_1_1_1_0      1
0_1_0_1_1      1
Name: count, dtype: int64
Следует ли выполнить стратифицированное раздеение на обучающую и тестовую выборку? y/n
Разделение датасета выполнено успешно


In [168]:
from sklearn.multioutput import MultiOutputClassifier


lgbm_model = MultiOutputClassifier(lgb.LGBMClassifier(verbose=-1))

columns_to_delete = []  

unique_classes = len(set(y_train))
print(unique_classes)

model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    #objective='binary',  
    #metric='binary_error' 
)

5


In [169]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics,base_multitarget=True)

evaluator.fit(X_train, y_train)

ModelEvaluator(base_multitarget=True,
               metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=['Label',
                                                                  'Brand'],
                                            columns_to_delete=[],
                                            model=MultiOutputClassifier(estimator=LGBMClassifier(verbose=-1))))

In [170]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,accuracy,precision,recall,f1_macro,Targets
0,0.696833,0.667706,0.62296,0.626322,Combination
1,0.674208,0.667859,0.632559,0.631803,Dry
2,0.714932,0.712557,0.635465,0.637771,Normal
3,0.665158,0.661243,0.620187,0.616103,Oily
4,0.638009,0.641355,0.638009,0.635854,Sensitive


In [171]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,accuracy,precision,recall,f1_macro,Targets
0,0.696833,0.667706,0.62296,0.626322,Combination
1,0.674208,0.667859,0.632559,0.631803,Dry
2,0.714932,0.712557,0.635465,0.637771,Normal
3,0.665158,0.661243,0.620187,0.616103,Oily
4,0.638009,0.641355,0.638009,0.635854,Sensitive


In [172]:
	accuracy	precision	recall	f1_macro	Targets
0	0.701357	0.669822	0.643939	0.64951	Combination
1	0.674208	0.660261	0.648618	0.650991	Dry
2	0.692308	0.662096	0.638465	0.643184	Normal
3	0.674208	0.661968	0.645208	0.64734	Oily
4	0.671946	0.672969	0.671946	0.67146	Sensitive

SyntaxError: invalid syntax (959147575.py, line 1)